## Evaluate

In [3]:
from datasets import load_dataset
import pandas as pd
pd.set_option('display.max_colwidth', 0)

## Create Open Assistant Evaluation Dataset

In [4]:
ds = load_dataset('OpenAssistant/oasst1')
ds

Using custom data configuration OpenAssistant--oasst1-fe5c583d2314bdd6
Found cached dataset parquet (/home/charipol/.cache/huggingface/datasets/OpenAssistant___parquet/OpenAssistant--oasst1-fe5c583d2314bdd6/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    validation: Dataset({
        features: ['message_id', 'parent_id', 'user_id', 'created_date', 'text', 'role', 'lang', 'review_count', 'review_result', 'deleted', 'rank', 'synthetic', 'model_name', 'detoxify', 'message_tree_id', 'tree_state', 'emojis', 'labels'],
        num_rows: 4401
    })
    train: Dataset({
        features: ['message_id', 'parent_id', 'user_id', 'created_date', 'text', 'role', 'lang', 'review_count', 'review_result', 'deleted', 'rank', 'synthetic', 'model_name', 'detoxify', 'message_tree_id', 'tree_state', 'emojis', 'labels'],
        num_rows: 84437
    })
})

In [60]:
train_df = pd.DataFrame(ds['train'])
train_df['split'] = 'train'
valid_df = pd.DataFrame(ds['validation'])
valid_df['split'] = 'valid'
all_df = pd.concat([train_df,valid_df],axis=0).reset_index(drop=True)
all_df.shape

(88838, 19)

In [61]:
all_df.groupby('lang').message_tree_id.nunique().reset_index()\
    .sort_values('message_tree_id', ascending=False)

,lang,message_tree_id
7,es,3915
6,en,3765
18,ru,766
24,zh,386
5,de,351
10,fr,263
2,ca,250
20,th,167
17,pt-BR,165
13,it,115


In [80]:
all_df['rnk'] = all_df.sort_values(['created_date'], ascending=[True]) \
             .groupby(['message_tree_id']) \
             .cumcount()

def get_nb(d, label):
    if d is None: return 0
    if (label in d['name']):
        idx = d['name'].index(label)
    else:
        return 0
    return d['count'][idx]
    
all_df['nb_plus'] = all_df.emojis.map(lambda d: get_nb(d, '+1'))
all_df['nb_minus'] = all_df.emojis.map(lambda d: get_nb(d, '-1'))
all_df['nb_net_plus'] = all_df.nb_plus - all_df.nb_minus

In [81]:
prompt_df = all_df[(all_df.rnk==0)][['lang',
                                     'message_tree_id',
                                     'message_id',
                                     'text',
                                     'nb_net_plus']]
prompt_df = prompt_df[prompt_df.nb_net_plus>=0]
prompt_df.shape, prompt_df.lang.value_counts()

((9714, 5),
 es       3667
 en       3449
 ru       720 
 zh       339 
 de       326 
 fr       241 
 ca       239 
 th       166 
 pt-BR    152 
 it       103 
 uk-UA    73  
 eu       65  
 ja       42  
 pl       38  
 vi       37  
 hu       17  
 ar       14  
 da       11  
 ko       5   
 tr       5   
 id       2   
 fi       2   
 cs       1   
 Name: lang, dtype: int64)

In [82]:
answer_df = all_df[(all_df.rnk>0)][['lang',
                                     'message_tree_id',
                                     'parent_id',
                                     'message_id',
                                     'text',
                                     'nb_net_plus']]
answer_df = answer_df[answer_df.nb_net_plus>=0]
answer_df['answer_rnk'] = answer_df.sort_values(['nb_net_plus'], ascending=[False]) \
             .groupby(['parent_id']) \
             .cumcount()
answer_df = answer_df[answer_df.answer_rnk==0]
answer_df.parent_id.value_counts(), answer_df.shape

(c8e83833-ecbc-44fe-b6db-735228c25a1c    1
 bc3c7784-1f2b-46f2-85e6-d27f3c264be2    1
 4782cd35-b062-4040-bf54-3a2f044e03e8    1
 b6f00415-1469-46a3-b425-be49f1533787    1
 6ecd3b03-7b02-46e4-aa6c-1aaf31ca03e7    1
                                        ..
 a436b070-34e6-405e-8c55-b066c5b12e98    1
 975f47da-300a-4f45-87b7-492a89cd299e    1
 ccb34f9c-6ac9-404e-9b33-9ac594c89be3    1
 ca517fbb-d007-40c6-85f4-5c354c51581b    1
 88ad2577-484c-4e87-b4d2-e12665881012    1
 Name: parent_id, Length: 40307, dtype: int64,
 (40307, 7))

In [85]:
qa_df  = prompt_df.merge(answer_df[['parent_id','text','nb_net_plus']], 
                         left_on='message_id', right_on='parent_id').reset_index(drop=True)
qa_df.columns = ['lang','message_tree_id','message_id',
                 'prompt','prompt_nb_net_plus','parent_id',
                 'answer','answer_nb_net_plus']
qa_df = qa_df[['lang','message_tree_id','message_id','parent_id',
               'prompt_nb_net_plus','answer_nb_net_plus','prompt','answer']]
qa_df.lang.value_counts()

es       3656
en       3425
ru       708 
zh       339 
de       324 
ca       239 
fr       238 
th       166 
pt-BR    152 
it       103 
uk-UA    73  
eu       65  
ja       42  
pl       37  
vi       37  
hu       17  
ar       14  
da       11  
ko       5   
tr       5   
id       2   
fi       2   
cs       1   
Name: lang, dtype: int64

In [91]:
#choose only sizeable languages of interest
lang_of_interest = ['th','ja','vi']
selected_df = qa_df[qa_df.lang.map(lambda x: x in lang_of_interest)]\
                    .sort_values('lang').reset_index(drop=True)
selected_df.lang.value_counts()

th    166
ja    42 
vi    37 
Name: lang, dtype: int64

In [94]:
selected_df[selected_df.lang=='th'].head(20)

,lang,message_tree_id,message_id,parent_id,prompt_nb_net_plus,answer_nb_net_plus,prompt,answer
42,th,8a512668-adeb-4533-8ba3-eb93aa802e09,8a512668-adeb-4533-8ba3-eb93aa802e09,8a512668-adeb-4533-8ba3-eb93aa802e09,2,4,ลืมรหัสผ่าน Discord ของตัวเอง ทำยังไงดี,ในหน้า Login คุณกรอก email หรือ เบอร์โทร แล้วลองกด Forget your Password? \nDiscord จะส่ง Email ไปหาคุณถ้าคุณพิมพ์ Email หรือ OTP ถ้าคุณกรอกเบอร์โทร ให้ทำตามขั้นตอนที่ Discord แจ้งคุณมาต่อไป\nหากคุณเกิดปัญหาอะไรขึ้นระหว่างทำตามขั้นตอนนั้นคุณสามารถกลับมาถามฉันให่ได้
43,th,3230604e-6d99-4990-b238-4e35d8369ab6,3230604e-6d99-4990-b238-4e35d8369ab6,3230604e-6d99-4990-b238-4e35d8369ab6,1,2,ร้านที่ขายข้าวหมูเกาหลี ยากิโซบะที่ศูนย์อาหารเดอะมอลล์โคราชย้ายไปที่ไหน หรือปิดไปแล้วครับ,ฉันขอโทษด้วย เนื่องจากฉันเป็นโมเดลภาษา ฉันไม่มีข้อมูลเกี่ยวกับสถานที่ต่าง ๆ ที่ปิดตัวหรือย้ายไปแล้วที่มีความถูกต้องและเป็นปัจจุบัน\n\nหากคุณต้องการข้อมูล คุณสามารถติดต่อศูนย์การค้า ค้นหาข้อมูลสถานที่เพิ่มเติมได้จากอินเทอร์เน็ต หรือสอบถามกลุ่มพูดคุยในอินเทอร์เน็ตที่เกี่ยวกับร้านอาหารในพื้นที่ที่คุณอยู่
44,th,5596a395-c2a8-4e19-ada8-b7653aa1161b,5596a395-c2a8-4e19-ada8-b7653aa1161b,5596a395-c2a8-4e19-ada8-b7653aa1161b,1,3,คิดว่ามีโอกาสที่จะเกิดสงครามระหว่างสหรัฐกับจีนในอีก 2-5 ปีข้างหน้ามั้ย,การทำนายเกี่ยวกับการเกิดสงครามระหว่างสหรัฐกับจีนในอนาคตไม่สามารถทำได้อย่างแม่นยำ โดยเฉพาะอย่างยิ่งในช่วงเวลาสั้นๆ เช่น 2-5 ปีข้างหน้า เนื่องจากมีหลายปัจจัยที่ส่งผลต่อการเกิดสงครามระหว่างประเทศสองแห่งนี้ เช่น ปัจจัยเศรษฐกิจ การเมือง ปัจจัยทางสังคม และปัจจัยทางทหาร เป็นต้น ดังนั้น การทำนายเกี่ยวกับการเกิดสงครามระหว่างสหรัฐกับจีนยังมีความไม่แน่นอนอยู่เสมอ\n\nอย่างไรก็ตาม สถานการณ์ระหว่างสหรัฐกับจีนในปัจจุบันยังคงมีความเสี่ยงที่จะเกิดขึ้นและทำให้สองประเทศมาอยู่ในสถานการณ์ที่ด้อยกว่าสถานการณ์ปัจจุบัน แต่การทำนายเกี่ยวกับการเกิดสงครามก็ไม่ควรเป็นเป้าหมายหลัก เนื่องจากการเตรียมความพร้อมและการสร้างความเข้าใจและการทำงานร่วมกันระหว่างสหรัฐกับจีนยังคงเป็นทางเลือกที่ดีที่สุดในการป้องกันการเกิดสงครามระหว่างสองประเทศในอนาคต
45,th,37b86e72-d229-4c48-a2a8-37f8716e3177,37b86e72-d229-4c48-a2a8-37f8716e3177,37b86e72-d229-4c48-a2a8-37f8716e3177,2,4,เผลอนอกใจเเฟน เเต่อยากได้เขากลับมาทำไงดี,แสดงเจตนารมณ์ของคุณด้วยการขอโทษอย่างจริงใจ และอดทนรอแฟนให้อภัย แต่ต้องสำนึกไว้เสมอว่า แฟนอาจจะไม่กลับมาแล้ว หรือความรักของคุณจะไม่เหมือนเดิมอีกต่อไป
46,th,fc7d2e64-b334-496f-a74c-19b2ee786652,fc7d2e64-b334-496f-a74c-19b2ee786652,fc7d2e64-b334-496f-a74c-19b2ee786652,3,1,อธิบายหลักการทำงานของเครื่องฟอกอากาศ ขอแบบที่เด็กห้าขวบเข้าใจ,เครื่องฟอกอากาศจะดูดอากาศที่มีฝุ่นปะปนอยู่ มาผ่านตาข่ายขนาดเล็กมากๆ เล็กพอที่จะดักเม็ดฝุ่นไว้ \nแล้วอากาศสะอาดก็จะถูกปล่อยออกไป โดยไม่มีฝุ่น\nแต่ถ้าฝุ่นเล็กกว่ารูตาข่าย ก็จะยังหลุดรอดออกไปได้
47,th,08cf8e85-8ad6-44a0-a418-b28fea3f8cfb,08cf8e85-8ad6-44a0-a418-b28fea3f8cfb,08cf8e85-8ad6-44a0-a418-b28fea3f8cfb,0,0,เราสามารถใส่อิเล็กตรอนขั่วบวกหรือลบ เข้าไปในวัตถุหรือร่างกายคนได้หรือเปล่าครับ,ไม่ได้ เพราะอิเล็กตรอนถือเป็นขั้วลบ ไม่ทราบว่าคุณหมายความว่าอะไร ฉันขอขยายความเพิ่มเติม
48,th,2f9530bd-7867-455c-8a1b-7644b9d0c61c,2f9530bd-7867-455c-8a1b-7644b9d0c61c,2f9530bd-7867-455c-8a1b-7644b9d0c61c,1,2,ความตายน่ากลัวหรือไม่,ไม่น่ากลัวเลย ความตายย่อมเป็นเรื่องของธรรมชาติ
49,th,7f387170-6706-4e5b-bfd3-aa1f365ab28f,7f387170-6706-4e5b-bfd3-aa1f365ab28f,7f387170-6706-4e5b-bfd3-aa1f365ab28f,0,2,ขอคำแนะนำคะ ซื้อของขวัญจับฉลากอะไรดี งบ 400-500 บาทค่ะ,ในช่วงราคานี้มีหลายอย่างที่เป็นที่นิยม เช่น ตุ๊กตา กิ๊ฟการ์ด ของกุ๊กกิ๊ก ฯลฯ
50,th,763ef1eb-e3a7-40bc-bf20-df0737510dbb,763ef1eb-e3a7-40bc-bf20-df0737510dbb,763ef1eb-e3a7-40bc-bf20-df0737510dbb,5,2,ปิดการใช้งาน accessibility service ใน android ยังไงครับ,บริการการช่วยเหลือการเข้าถึงอาจเป็นตัวเลือกที่ดีหากคุณเป็นผู้พิการและต้องการใช้อุปกรณ์ของคุณง่ายขึ้น แต่ในขณะเดียวกันก็สร้างความกังวลว่าจะเปิดช่องให้มิจฉาชีพเข้าถึงอุปกรณ์ของคุณในทางที่ไม่ดีได้\n\nคุณสามารถทำตามขั้นตอนดังนี้เพื่อปิดบริการการช่วยเหลือการเข้าถึงในระบบ Android:\n\n1. เข้าไปที่ การตั้งค่า ในอุปกรณ์ของคุณ\n2. ขึ้นอยู่กับอุปกรณ์ของคุณ เข้าไปที่เมนูใดเมนูหนึ่งตามนี้:\n 1. การตั้งค่าเพิ่มเติม จากนั้นไปที่ การช่วยเหลือพิเศษ หรือ\n 2. การช่วยเหลือก

## Get Answer from `text-davinci-003`

In [234]:
import os
import openai

credentials = pd.read_csv('../../credentials/openai.csv')
openai.organization = credentials['organization'][0]
openai.api_key = credentials['api_key'][0]

In [235]:
def get_openai_answer(prompt):
    response = openai.Completion.create(
      model="text-davinci-003",
      prompt=prompt,
      max_tokens=1024,
      temperature=0,
      top_p=1.0,
      frequency_penalty=0.0,
      presence_penalty=0.0,
    )
    return response['choices'][0]['text'].split('\n\n')[-1]

In [ ]:
from tqdm.auto import tqdm

for i in tqdm(selected_df.itertuples()):
    prompt = i[7]
    selected_df.loc[i[0], 'answer_davinci003'] = get_openai_answer(prompt)
    
selected_df.to_csv('../data_large/oasst1_benchmark.csv', index=False)

0it [00:00, ?it/s]

## Evaluate Answer Pairs

###  `text-davinci-003`

In [ ]:
#modified vicuna eval templates
vicuna_template = {"prompt_id": 1, 
                   "system_prompt": "You are a helpful and precise assistant for checking the quality of the answer.", 
                   "prompt_template": "[Question]\n{question}\n\n[The Start of Assistant 1's Answer]\n{answer_1}\n\n[The End of Assistant 1's Answer]\n\n[The Start of Assistant 2's Answer]\n{answer_2}\n\n[The End of Assistant 2's Answer]\n\n[System]\n{prompt}\n\n", 
                   "defaults": {
                       "prompt": "We would like to request your feedback on the performance of two AI assistants in response to the user question displayed above.\nPlease rate the helpfulness, relevance, accuracy, level of details of their responses. Each assistant receives an overall score on a scale of 0 to 1, where a higher score indicates better overall performance.\nPlease first output a single line containing only two values indicating the scores for Assistant 1 and 2, respectively. The two scores are separated by a space. In the subsequent line, please provide a comprehensive explanation of your evaluation, avoiding any potential bias and ensuring that the order in which the responses were presented does not affect your judgment."
                   }, 
                   "description": "Prompt for general questions", 
                   "category": "general"}

In [ ]:
for i in tqdm(selected_df.itertuples()):
    ques = selected_df.loc[i[0],'prompt']
    ans1 = selected_df.loc[i[0],'answer']
    ans2 = selected_df.loc[i[0],'answer_davinci003']

    prompt_template = vicuna_template["prompt_template"]
    user_prompt = prompt_template.format(
        question=ques, answer_1=ans1, answer_2=ans2, **vicuna_template["defaults"]
    )
    
    response = openai.Completion.create(
        model="text-davinci-003",
        prompt= f'{vicuna_template["system_prompt"]} {user_prompt}',
        max_tokens=1024,
        temperature=0.2,
        top_p=1.0,
        frequency_penalty=0.0,
        presence_penalty=0.0,
    )
    
    selected_df.loc[i[0],'score_human_davinci003'] = response['choices'][0]['text'].strip()

selected_df.to_csv('../data_large/oasst1_benchmark.csv', index=False)

In [ ]:
selected_df

### WangchangLM-sft-en-only

In [ ]:
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
)
from peft import (
    PeftModel,
)
from typing import Optional, Union, List, Dict, Any
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import torch
import re

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('facebook/xglm-7.5B')
#load merged version
model = AutoModelForCausalLM.from_pretrained(
    'pythainlp/wangchanglm-7.5B-sft-adapter-merged',
model.eval()
model.get_memory_footprint()

In [ ]:
def infer_vicuna(text):
    input_text = f"<human>: {text} <bot>: "
    batch = tokenizer(input_text, return_tensors='pt')
    output_tokens = model.generate(**batch, 
                                   no_repeat_ngram_size=2,
                                   num_beams=5,
                                   min_length=len(batch['input_ids'])+64,
                                   max_new_tokens=512,
#                                    begin_suppress_tokens = exclude_ids,
    #                                logits_processor=[logits_processor],
                                  )
    return tokenizer.decode(output_tokens[0], skip_special_tokens=True)

In [ ]:
for i in tqdm(range(vicuna_test.shape[0])):
    vicuna_test.loc[i,'a_answer'] = infer_vicuna(
        vicuna_test.loc[i,'text_th']
    ).split('<bot>: ')[-1]
    
vicuna_test.to_csv('../data/vicuna_test_a.csv',index=False)

### WangchangLM-sft-en-th

### WangchangLM-sft-en-th-rlhf